In [1]:
from pdp_utils import *
import random
import numpy as np
import functools

np.random.seed(20)

def resolve_max_capacity(prob, vehicle_idx):
    capacities = prob["VesselCapacity"]
    try:
        return capacities[vehicle_idx]
    # Index out of bounds: give dummy truck infinite capacity
    except:
        return float('inf')
    
def resolve_allowed_calls(prob, vehicle_idx):
    allowed_calls_list = prob["VesselCargo"]
    try:
        return allowed_calls_list[vehicle_idx]
    # Index out of bounds, allow dummy truck all calls
    except:
        return np.array([1. for i in range(prob["n_calls"])])

    


In [2]:
class Truck:
    
    def __init__(self, idx, max_capacity, allowed_calls):
        self.idx = idx
        self.max_capacity = max_capacity
        self.allowed_calls = allowed_calls
        self.position = -1
        self.time = 0
        self.carrying = 0
        self.cost = 0
        self.actions = []
        
    def take_call(self, call_idx, prob):
        # Assume truck taking the call will pick up good and immediately drive to destination, 
        # not picking up another call on the way.
        self.time = self.resolve_complete_call_time(call_idx, prob)
        self.cost += self.resolve_complete_call_cost(call_idx, prob)
        self.position = int(prob["Cargo"][call_idx][1]) - 1
        self.actions.append(call_idx)
        pass
    
    def resolve_complete_call_time(self, call_idx, prob):
        node_from = int(prob["Cargo"][call_idx][0]) - 1
        node_to = int(prob["Cargo"][call_idx][1]) - 1
        if self.time == 0:
            time_to = prob["FirstTravelTime"][self.idx][node_from]
        else:
            time_to = prob["TravelTime"][self.idx][self.position][node_from]
        time_to = max(self.time + time_to, prob["Cargo"][call_idx][4])
        time_to += prob["LoadingTime"][self.idx][call_idx]
        time_to += prob["TravelTime"][self.idx][node_from][node_to]
        time_to = max(self.time + time_to, prob["Cargo"][call_idx][6])
        time_to += prob["LoadingTime"][self.idx][call_idx]
        return time_to
    
    def resolve_complete_call_cost(self, call_idx, prob):
        node_from = int(prob["Cargo"][call_idx][0]) - 1
        node_to = int(prob["Cargo"][call_idx][1]) - 1
        if self.cost == 0:
            cost_to = prob["FirstTravelCost"][self.idx][node_from]
        else:
            cost_to = prob["TravelCost"][self.idx][self.position][node_from]
        cost_from = prob["TravelCost"][self.idx][node_from][node_to]
        return cost_to + cost_from
    
    def reject_call(self, call_idx, prob):
        self.actions.append(call_idx)
        # Add cost of not transporting
        self.cost += prob["Cargo"][call_idx][3]
        

In [3]:
def generate_trucks(prob):
    trucks = []
    for i in range(prob["n_vehicles"]):
        initial_call = np.random.choice(np.size(prob["Cargo"], 0))
        max_capacity = resolve_max_capacity(prob, i)
        allowed_calls = resolve_allowed_calls(prob, i)    
        truck = Truck(i, max_capacity, allowed_calls)
        trucks.append(truck)
    dummy_truck = Truck(prob["n_vehicles"], float('inf'), np.array([1 for i in range(len(prob["VesselCargo"][0]))]))
    return trucks, dummy_truck

In [4]:
def run_problem(problem):
    prob = load_problem(problem)
    cargos = prob["Cargo"].copy()
    trucks, dummy_truck = generate_trucks(prob)
    available_trucks = trucks.copy()
    while len(cargos) > 0:
        to_select = np.random.choice(np.size(cargos, 0))
        current_call = cargos[to_select]
        call_idx = np.where(np.all(prob["Cargo"] == current_call,axis=1))[0][0]

        # Filter trucks not allowed to take call
        available_trucks = [truck for truck in available_trucks \
                                if truck.allowed_calls[call_idx] == 1]
        # Filter trucks where time elapsed is too great
        available_trucks = [truck for truck in available_trucks \
                                if truck.time + truck.resolve_complete_call_time(call_idx, prob) < current_call[7] ]
        # Filter trucks where capacity is not sufficient
        available_trucks = [truck for truck in available_trucks \
                                if truck.max_capacity > current_call[2]]

        # If no trucks can make call, remove call from list and continue
        if len(available_trucks) < 1:
            dummy_truck.reject_call(call_idx, prob)
            cargos = np.delete(cargos, to_select, axis=0)
            continue

        # Some truck can make the call: Select random truck among available trucks.
        truck = random.choice(available_trucks)
        truck.take_call(call_idx, prob)
        
    solution = functools.reduce(lambda acc, val: acc + [i + 1 for i in val.actions] + [0], trucks, [])
    solution = solution + [i+1 for i in dummy_truck.actions]
    feasiblity, c = feasibility_check(solution, prob)
    sol_cost = cost_function(solution, prob)
    sol_cost += dummy_truck.cost
    return solution, sol_cost, feasiblity, c, prob
    
    

## C7V3

In [5]:
import time
n = 1000

start_time = time.time()
c7v3_init_sol, c7v3_init_cost, feasibility, c, prob = run_problem('./Call_7_Vehicle_3.txt')
c7v3_best_sol = c7v3_init_sol
c7v3_best_cost = c7v3_init_cost
c7v3_total_cost = c7v3_init_cost
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_7_Vehicle_3.txt')
    c7v3_total_cost += sol_cost
    if(sol_cost < c7v3_best_cost):
        c7v3_best_sol = solution
        c7v3_best_cost = sol_cost
end_time = time.time()

c7v3_avg_cost = c7v3_total_cost / (n+1)
print(c7v3_best_sol)
print('init_cost', cost_function(c7v3_init_sol, prob))
print('best_cost', c7v3_best_cost)
print('avg_cost', c7v3_avg_cost)
print('improvement', ((c7v3_init_cost - c7v3_best_cost) / c7v3_init_cost)*100)
print('elapsed time', end_time - start_time)

[0, 0, 3, 0, 1, 6, 3, 7, 5, 2, 4]
init_cost 1908673.0
best_cost 4912253.5
avg_cost 5022190.367132867
improvement 4.6404711977447235
elapsed time 43.92014217376709


## C18V5

In [6]:

start_time = time.time()
c18v5_init_sol, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
c18v5_init_cost = cost_function(c18v5_init_sol, prob)
c18v5_best_sol = c18v5_init_sol
c18v5_best_cost = c18v5_init_cost
c18v5_total_cost = c18v5_init_cost
for i in range(n):
    solution, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
    sol_cost = cost_function(solution, prob)
    c18v5_total_cost += sol_cost
    if(sol_cost < c18v5_best_cost):
        c18v5_best_sol = solution
        c18v5_best_cost = sol_cost
end_time = time.time()

c18v5_avg_cost = c18v5_total_cost / (n+1)
print(c18v5_best_sol)
print('init_cost', cost_function(c18v5_init_sol, prob))
print('best_cost', c18v5_best_cost)
print('avg_cost', c18v5_avg_cost)
print('improvement', ((c18v5_init_cost - c18v5_best_cost) / c18v5_init_cost)*100)
print('elapsed time', end_time - start_time)

ValueError: too many values to unpack (expected 4)

## C35V7 

In [ ]:
start_time = time.time()
c35v7_init_sol, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
c35v7_init_cost = cost_function(c35v7_init_sol, prob)
c35v7_best_sol = c35v7_init_sol
c35v7_best_cost = c35v7_init_cost
c35v7_total_cost = c35v7_init_cost
for i in range(n):
    solution, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
    sol_cost = cost_function(solution, prob)
    c35v7_total_cost += sol_cost
    if(sol_cost < c35v7_best_cost):
        c35v7_best_sol = solution
        c35v7_best_cost = sol_cost
end_time = time.time()

c35v7_avg_cost = c35v7_total_cost / (n+1)
print(c35v7_best_sol)
print('init_cost', cost_function(c35v7_init_sol, prob))
print('best_cost', c35v7_best_cost)
print('avg_cost', c35v7_avg_cost)
print('improvement', ((c35v7_init_cost - c35v7_best_cost) / c35v7_init_cost)*100)
print('elapsed time', end_time - start_time)

## C80V20

In [ ]:
start_time = time.time()
c80v20_init_sol, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
c80v20_init_cost = cost_function(c80v20_init_sol, prob)
c80v20_best_sol = c80v20_init_sol
c80v20_best_cost = c80v20_init_cost
c80v20_total_cost = c80v20_init_cost
for i in range(n):
    solution, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
    sol_cost = cost_function(solution, prob)
    c80v20_total_cost += sol_cost
    if(sol_cost < c80v20_best_cost):
        c80v20_best_sol = solution
        c80v20_best_cost = sol_cost
end_time = time.time()

c80v20_avg_cost = c80v20_total_cost / (n+1)
print(c80v20_best_sol)
print('init_cost', cost_function(c80v20_init_sol, prob))
print('best_cost', c80v20_best_cost)
print('avg_cost', c80v20_avg_cost)
print('improvement', ((c80v20_init_cost - c80v20_best_cost) / c80v20_init_cost)*100)
print('elapsed time', end_time - start_time)

## C130V40

In [ ]:
start_time = time.time()
c130v40_init_sol, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
c130v40_init_cost = cost_function(c130v40_init_sol, prob)
c130v40_best_sol = c130v40_init_sol
c130v40_best_cost = c130v40_init_cost
c130v40_total_cost = c130v40_init_cost
for i in range(n):
    solution, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
    sol_cost = cost_function(solution, prob)
    c130v40_total_cost += sol_cost
    if(sol_cost < c130v40_best_cost):
        c130v40_best_sol = solution
        c130v40_best_cost = sol_cost
end_time = time.time()

c130v40_avg_cost = c130v40_total_cost / (n+1)
print(c130v40_best_sol)
print('init_cost', cost_function(c130v40_init_sol, prob))
print('best_cost', c130v40_best_cost)
print('avg_cost', c130v40_avg_cost)
print('improvement', ((c130v40_init_cost - c130v40_best_cost) / c130v40_init_cost)*100)
print('elapsed time', end_time - start_time)

## C300V90

In [ ]:
start_time = time.time()
c300v90_init_sol, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
c300v90_init_cost = cost_function(c300v90_init_sol, prob)
c300v90_best_sol = c300v90_init_sol
c300v90_best_cost = c300v90_init_cost
c300v90_total_cost = c300v90_init_cost
for i in range(n):
    solution, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
    sol_cost = cost_function(solution, prob)
    c300v90_total_cost += sol_cost
    if(sol_cost < c300v90_best_cost):
        c300v90_best_sol = solution
        c300v90_best_cost = sol_cost
end_time = time.time()

c300v90_avg_cost = c300v90_total_cost / (n+1)
print(c300v90_best_sol)
print('init_cost', cost_function(c300v90_init_sol, prob))
print('best_cost', c300v90_best_cost)
print('avg_cost', c300v90_avg_cost)
print('improvement', ((c300v90_init_cost - c300v90_best_cost) / c300v90_init_cost)*100)
print('elapsed time', end_time - start_time)

In [ ]:
print(dummy_truck)